In [1]:
import os
import time
import numpy as np
import tensorflow as tf
import librosa
import librosa.display
import soundfile as sf
import pyaudio
from tensorflow.keras.models import load_model
import IPython.display as ipd
import noisereduce as nr

In [2]:
# Définition des paramètres
chunk = 1024
format = pyaudio.paInt16
channels = 1
rate = 16000
record_seconds = 15
threshold = 0.1
silence_seconds = 7

# Define the CTC loss function
def CTCLoss(y_true, y_pred):
    input_length = tf.math.reduce_sum(tf.ones_like(y_pred), axis=1)
    label_length = tf.math.reduce_sum(tf.ones_like(y_true), axis=1)
    loss = tf.keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss 

# Load the pre-trained model
try:
    with tf.keras.utils.custom_object_scope({'CTCLoss': CTCLoss}):
        model = tf.keras.models.load_model('voice_in_typing.h5')
        print(model)
except:
    print("Error loading the pre-trained model")

In [6]:
# Fonction pour enregistrer le son à partir du microphone
def record_audio(record_seconds):
    p = pyaudio.PyAudio()
    stream = p.open(format=format,
                    channels=channels,
                    rate=rate,
                    input=True,
                    frames_per_buffer=chunk)

    frames = []

    print("Enregistrement en cours...")
    for i in range(0, int(rate / chunk * record_seconds)):
        data = stream.read(chunk)
        frames.append(data)

    print("Enregistrement terminé.")

    stream.stop_stream()
    stream.close()
    p.terminate()

    # Conversion des données audio en tableau numpy
    audio = np.frombuffer(b''.join(frames), dtype=np.int16)

    # Normalisation du son
    audio_norm = audio / (np.max(np.abs(audio)) + 1e-7)

    return audio_norm

# Fonction pour détecter le début et la fin du son
def detect_sound(audio_norm, threshold, silence_seconds):
    start = None
    end = None

    for i in range(len(audio_norm)):
        if abs(audio_norm[i]) > threshold:
            start = i
            break

    if start is not None:
        for i in range(len(audio_norm) - 1, 0, -1):
            if abs(audio_norm[i]) > threshold:
                end = i
                break

    if end is not None:
        for i in range(end, 0, -1):
            if abs(audio_norm[i]) < threshold:
                if end - i > silence_seconds * rate:
                    end = i
                    break

    return start, end

# Fonction pour supprimer le fichier audio
def delete_audio(filename):
    if os.path.exists(filename):
        os.remove(filename)
        

# Enregistrement et reconnaissance du son
audio_norm = record_audio(record_seconds)

# Détection du début et de la fin du son
start, end = detect_sound(audio_norm, threshold, silence_seconds)

if start is not None and end is not None:
    audio_norm = audio_norm[start:end]

    # Enregistrement du son dans un fichier
    filename = 'audio.wav'
    sf.write(filename, audio_norm, rate)

    # Lecture du fichier audio
    audio, sr = librosa.load(filename, sr=rate, mono=True)


Enregistrement en cours...
Enregistrement terminé.


In [7]:
ipd.Audio('audio.wav')

In [8]:
import speech_recognition as sr

    # Créer un objet Recognizer pour utiliser les fonctionnalités de la bibliothèque SpeechRecognition
r = sr.Recognizer()

    # Spécifier le nom et le chemin d'accès du fichier audio à transcrire
audio_file = sr.AudioFile('audio.wav')

    # Ouvrir le fichier audio pour la lecture
with audio_file as source:
    # Écouter l'audio pour identifier les segments de discours
      audio = r.record(source)

    # Transcrire le discours identifié dans le fichier audio en texte à l'aide de l'API Google Speech Recognition
      text = r.recognize_google(audio)

    # Afficher le texte transcrit
      print("Prediction:", text)

if text:
    print(f"Texte reconnu: {text}")
    # Appel de la fonction pour supprimer le fichier audio
    delete_audio(filename)
   
else:
    print("Aucun texte détecté")
    # Appel de la fonction pour supprimer le fichier audio
    delete_audio(filename)
    


Prediction: miss you
Texte reconnu: miss you
